In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import pandas as pd
import pickle
import numpy as np
import re


In [3]:
# 파일 불러오기
with open('../../data/df_tra.pkl', 'rb') as f:
    df_tra = pickle.load(f)


In [4]:
# 파일 불러오기
with open('../../data/df_tra_ans.pkl', 'rb') as f:
    df_tra_ans = pickle.load(f)


In [ ]:
# 파일 불러오기
with open('../../data/df_tra_ans_acc.pkl', 'rb') as f:
    df_tra_ans_acc = pickle.load(f)
    

In [6]:
rel_path = '../LLM/result/'

In [7]:
df_tra.head()

,ver,q_id,u_id,q_creationdate,u_creationdate,reputation
0,250000,70477687,16801971,2021-12-25,2021-09-01 05:23:37.143,20
1,250000,70477822,6942666,2021-12-25,2016-10-08 20:15:01.120,2819
2,250000,70478253,16684888,2021-12-25,2021-08-17 04:24:20.873,30
3,250000,70478356,2741711,2021-12-25,2013-09-03 04:56:44.190,193
4,250000,70478466,17759657,2021-12-25,2021-12-25 06:53:56.140,46


In [ ]:
df_tra['q_creationdate'] = pd.to_datetime(df_tra['q_creationdate'])
df_tra['u_creationdate'] = pd.to_datetime(df_tra['u_creationdate'])


df_tra_ans['a_creationdate'] = pd.to_datetime(df_tra_ans['a_creationdate'])
df_tra_ans['a_u_creationdate'] = pd.to_datetime(df_tra_ans['a_u_creationdate'])


df_tra_ans_acc['a_creationdate'] = pd.to_datetime(df_tra_ans_acc['a_creationdate'])
df_tra_ans_acc['a_u_creationdate'] = pd.to_datetime(df_tra_ans_acc['a_u_creationdate'])

In [ ]:
df_tra['diff_days'] = [x.days for x in df_tra['q_creationdate'] - df_tra['u_creationdate']]
df_tra_ans['diff_days'] = [x.days for x in df_tra_ans['a_creationdate'] - df_tra_ans['a_u_creationdate']]
df_tra_ans_acc['diff_days'] = [x.days for x in df_tra_ans_acc['a_creationdate'] - df_tra_ans_acc['a_u_creationdate']]

In [10]:
df_tra.head()

,ver,q_id,u_id,q_creationdate,u_creationdate,reputation,diff_days
0,250000,70477687,16801971,2021-12-25,2021-09-01 05:23:37.143,20,114
1,250000,70477822,6942666,2021-12-25,2016-10-08 20:15:01.120,2819,1903
2,250000,70478253,16684888,2021-12-25,2021-08-17 04:24:20.873,30,129
3,250000,70478356,2741711,2021-12-25,2013-09-03 04:56:44.190,193,3034
4,250000,70478466,17759657,2021-12-25,2021-12-25 06:53:56.140,46,-1


In [11]:
def make_one_file(ver):
    path = f'{rel_path}{ver}'
    file_list = os.listdir(path)
    df = pd.DataFrame()
    if len(file_list)>0 : 
        for f in file_list:
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            df = pd.concat([df, tmp], axis =0)

        df.sort_values(by = ['creationdate']).reset_index(drop=True)
        return df
    else :
        return np.NAN

In [12]:
def pp_df(df, sc_num):
    df_copy = df.copy()
    df_copy['creationdate'] = pd.to_datetime(df_copy['creationdate'])
    df_c = df.copy()
    df_c = df_c[~df_c['result'].isna()]
    df_c['o_result'] = df_c['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
    df_c = df_c[df_c['o_result'].isin(['1', '0', '2'])]
    
    df_c.loc[:, 'cnt'] = 1
    chk_cnt = df_c.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'cnt']]
    chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]


    m_chk_cnt = pd.merge(chk_cnt, df_copy, on = 'id')

    m_chk_cnt = m_chk_cnt[['ver', 'creationdate', 'id', 'o_result']].drop_duplicates()
        
    return m_chk_cnt
        



In [13]:
path = f'{rel_path}'
file_list = os.listdir(path)
sample_list = sorted([y for y in [x for x in file_list if x.isdigit()] if int(y)>=50000])

In [14]:
tot_calc = pd.DataFrame()
for listid in sample_list:
    df = make_one_file(listid)
    if isinstance(df, pd.core.frame.DataFrame):
        df = pp_df(df, 5)
        tot_calc = pd.concat([tot_calc, df], axis = 0)

tot_calc = tot_calc.reset_index(drop=True)

    

In [15]:
df_tra_ans.head()

,ver,q_id,q_creationdate,a_u_id,a_creationdate,a_u_creationdate,reputation,diff_days
0,250000,70477687,2021-12-25,2538939,2021-12-25 03:07:50.677,2013-07-01 12:17:50.170,77712,3098
1,250000,70477822,2021-12-25,6942666,2022-01-26 13:31:55.050,2016-10-08 20:15:01.120,3014,1935
2,250000,70478253,2021-12-25,1431750,2021-12-25 07:20:24.053,2012-06-01 23:17:33.137,11547,3493
3,250000,70478356,2021-12-25,12750353,2021-12-28 21:20:07.490,2020-01-20 20:10:22.017,3503,708
4,250000,70478466,2021-12-25,10809821,2021-12-25 07:10:01.873,2018-12-19 07:57:15.287,53,1101


In [ ]:
df_merge = pd.merge(df_tra, tot_calc, left_on = ['ver', 'q_id', 'q_creationdate'], right_on = ['ver', 'id', 'creationdate'])


df_merge_ans = pd.merge(df_tra_ans, tot_calc, left_on = ['ver', 'q_id', 'q_creationdate'], right_on = ['ver', 'id', 'creationdate'])

df_merge_ans_acc = pd.merge(df_tra_ans_acc, tot_calc, left_on = ['ver', 'q_id', 'q_creationdate'], right_on = ['ver', 'id', 'creationdate'])



In [17]:
# diff = pd.merge(tot_calc, df_merge[['ver', 'creationdate', 'id', 'u_id']], how = 'left', left_on = ['ver', 'creationdate', 'id'], right_on = ['ver', 'creationdate', 'id'])
# diff[diff['u_id'].isna()]
# users 테이블에  id 가 없는 경우 NaN으로 출력되는 경우 발생 (가입하지 않고도 글을 쓸 수 있나봄? 혹은 가입하고 글쓰고 나서 탈퇴하거나)

In [18]:
df_merge_ans.head()

,ver,q_id,q_creationdate,a_u_id,a_creationdate,a_u_creationdate,reputation,diff_days,creationdate,id,o_result
0,250000,70477687,2021-12-25,2538939,2021-12-25 03:07:50.677,2013-07-01 12:17:50.170,77712,3098,2021-12-25,70477687,1
1,250000,70477822,2021-12-25,6942666,2022-01-26 13:31:55.050,2016-10-08 20:15:01.120,3014,1935,2021-12-25,70477822,2
2,250000,70478253,2021-12-25,1431750,2021-12-25 07:20:24.053,2012-06-01 23:17:33.137,11547,3493,2021-12-25,70478253,1
3,250000,70478356,2021-12-25,12750353,2021-12-28 21:20:07.490,2020-01-20 20:10:22.017,3503,708,2021-12-25,70478356,2
4,250000,70478984,2021-12-25,109525,2021-12-26 17:50:23.053,2009-05-19 18:11:05.280,8706,4603,2021-12-25,70478984,1


In [ ]:
df_merge['signin'] = np.ceil(df_merge['diff_days']/365)
df_merge_ans['signin'] = np.ceil(df_merge_ans['diff_days']/365)
df_merge_ans_acc['signin'] = np.ceil(df_merge_ans_acc['diff_days']/365)

In [20]:
df_merge.head()

,ver,q_id,u_id,q_creationdate,u_creationdate,reputation,diff_days,creationdate,id,o_result,signin
0,250000,70477687,16801971,2021-12-25,2021-09-01 05:23:37.143,20,114,2021-12-25,70477687,1,1.0
1,250000,70477822,6942666,2021-12-25,2016-10-08 20:15:01.120,2819,1903,2021-12-25,70477822,2,6.0
2,250000,70478253,16684888,2021-12-25,2021-08-17 04:24:20.873,30,129,2021-12-25,70478253,1,1.0
3,250000,70478356,2741711,2021-12-25,2013-09-03 04:56:44.190,193,3034,2021-12-25,70478356,2,9.0
4,250000,70478925,13995674,2021-12-25,2020-07-25 22:41:24.933,0,517,2021-12-25,70478925,0,2.0


In [21]:
df_merge_ans.head()

,ver,q_id,q_creationdate,a_u_id,a_creationdate,a_u_creationdate,reputation,diff_days,creationdate,id,o_result,signin
0,250000,70477687,2021-12-25,2538939,2021-12-25 03:07:50.677,2013-07-01 12:17:50.170,77712,3098,2021-12-25,70477687,1,9.0
1,250000,70477822,2021-12-25,6942666,2022-01-26 13:31:55.050,2016-10-08 20:15:01.120,3014,1935,2021-12-25,70477822,2,6.0
2,250000,70478253,2021-12-25,1431750,2021-12-25 07:20:24.053,2012-06-01 23:17:33.137,11547,3493,2021-12-25,70478253,1,10.0
3,250000,70478356,2021-12-25,12750353,2021-12-28 21:20:07.490,2020-01-20 20:10:22.017,3503,708,2021-12-25,70478356,2,2.0
4,250000,70478984,2021-12-25,109525,2021-12-26 17:50:23.053,2009-05-19 18:11:05.280,8706,4603,2021-12-25,70478984,1,13.0


In [22]:
# 전체 데이터셋 
with open('../../data/df_tra_user.pkl', 'wb') as f:
    pickle.dump(df_merge, f)

In [23]:
# 전체 데이터셋 
with open('../../data/df_tra_ans_user.pkl', 'wb') as f:
    pickle.dump(df_merge_ans, f)


In [ ]:
# 전체 데이터셋 
with open('../../data/df_tra_ans_user_acc.pkl', 'wb') as f:
    pickle.dump(df_merge_ans_acc, f)
